In [1]:
import warnings
from itertools import product
import glob
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.path as mpath
import cartopy
import cartopy.crs as ccrs
import cartopy.feature
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import cartopy.feature as cf
import shapely.geometry as sgeom

from sklearn.decomposition import PCA
from scipy import stats
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

import pickle
import copy
from shapely import geometry
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
maxtemp = xr.open_dataset('/glade/derecho/scratch/jhayron/TMax_ERA5_Daily.nc').MaxTemp.compute()
## Mask ##

lsm = xr.open_dataset('/glade/campaign/collections/rda/data/d633000/e5.oper.invariant/197901/e5.oper.invariant.128_172_lsm.ll025sc.1979010100_1979010100.nc')
lsm = lsm.sel(latitude=maxtemp.lat,longitude=maxtemp.lon)
lsm = lsm.LSM.isel(time=0)
lsm = lsm.astype(bool)
mask_arr = lsm.data.astype(float)
mask_arr[mask_arr==False]=np.nan
data_arr = maxtemp.data
data_arr = data_arr*mask_arr
maxtemp.data = data_arr

In [3]:
indices_seasons = {'Winter':[12,1,2],
                   'Spring':[3,4,5],
                   'Summer':[6,7,8],
                   'Fall':[9,10,11],
                   'All':[1,2,3,4,5,6,7,8,9,10,11,12]}

In [4]:
import string
abcd = list(string.ascii_lowercase)

In [5]:
abcd2 = abcd+abcd

# p90th of temperature

In [6]:
maxtemp = maxtemp.sel(time=~((maxtemp['time.month'] == 2) & (maxtemp['time.day'] == 29)))
maxtemp['year'] = maxtemp['time.year']
maxtemp['dayofyear'] = ('time',np.tile(
        np.arange(1, 366, 1, dtype='int32'),
        len(pd.to_datetime(maxtemp.time).year.unique())
    ))

In [7]:
len(np.tile(
        np.arange(1, 366, 1, dtype='int32'),
        len(pd.to_datetime(maxtemp.time).year.unique())
    ))

30660

In [8]:
maxtemp_reshaped = maxtemp.set_index(time=('year', 'dayofyear')).unstack('time')

In [9]:
q90 = maxtemp_reshaped.sel(year=slice(1948,1977)).quantile(0.9,dim='year')

/glade/work/jhayron/conda-envs/weather_regimes/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [10]:
stacked = xr.concat([q90, q90, q90], dim='dayofyear')
stacked = stacked.rolling(dayofyear=5, center=True).mean()
q90_smooth = stacked.isel(dayofyear=slice(365, 365*2))

In [11]:
higherthanp90 = maxtemp_reshaped > q90_smooth
higherthanp90 = higherthanp90.stack(time=('year', 'dayofyear'))
higherthanp90 = higherthanp90.transpose('time', 'lat', 'lon').astype(int)

In [12]:
higherthanp90['time'] = maxtemp['time']

/glade/derecho/scratch/jhayron/tmp/ipykernel_35845/4006091584.py:1: FutureWarning: updating coordinate 'time' with a PandasMultiIndex would leave the multi-index level coordinates ['year', 'dayofyear'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['time', 'year', 'dayofyear'])` before assigning new coordinate values.
  higherthanp90['time'] = maxtemp['time']


In [13]:
higherthanp90.to_netcdf('/glade/derecho/scratch/jhayron/Data4WRsClimateChange/Temp/TMaxp90_ERA5_v2.nc')

# warm spells

In [14]:
def find_start_durations(data):
    """Finds durations of consecutive 1s and assigns them to start times."""
    durations = np.zeros_like(data, dtype=int)  # Initialize durations array
    for lat_idx in range(data.shape[1]):
        for lon_idx in range(data.shape[2]):
            time_series = data[:, lat_idx, lon_idx]
            if np.any(time_series):
                diffs = np.diff(np.concatenate(([0], time_series, [0])))
                starts = np.where(diffs == 1)[0]
                ends = np.where(diffs == -1)[0]
                for start, end in zip(starts, ends):
                    durations[start, lat_idx, lon_idx] = end - start
    return durations

In [15]:
# Assuming 'MaxTemp' is your xarray.DataArray
durations = find_start_durations(higherthanp90.values)

In [16]:
durations[durations<6] = 0
durations[durations>=6] = 1

In [17]:
durations_da = xr.DataArray(
    durations,
    coords=higherthanp90.coords,
    dims=higherthanp90.dims,
    name='start_durations'
)

# print(durations_da)

In [19]:
durations_da.to_netcdf('/glade/derecho/scratch/jhayron/Data4WRsClimateChange/Temp/warm_spells_v2.nc')

In [20]:
durations_da

<xarray.DataArray 'start_durations' (time: 30660, lat: 321, lon: 521)> Size: 41GB
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
...
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]])
Coordinates:
  * lat        (lat) float64 3kB 10.0 10.25 10.5 10.75 ... 89.25 89.5 89.75 90.0
  * lon        (lon) float64 4kB 180.0 180.2 180.5 180.8 ... 309.5 309.8 310.0
    quantile   float64 8B 0.9
  * time       (time) datetime64[ns] 245kB 1940-01-01 1940-01-02 ... 2023-12-31
    year       (time) int64 245kB 1940 1940 1940 1940 ... 2023 2023 2023 2023
    dayofyear  (time) int32 123kB 1 2 3 4 5 6 7 ... 359 360 361 362 363 364 365